#### We are focusing on Commericial Real Estate Properties in New York City

In [1]:
# Import Libraries

import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("assets/nyc-rolling-sales.csv")

df.head(5)

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [4]:
for category in df['BUILDING CLASS CATEGORY'].unique():
    count = df[df['BUILDING CLASS CATEGORY'] == category].shape[0]
    print(f"{category}: {count}")

07 RENTALS - WALKUP APARTMENTS             : 3466
08 RENTALS - ELEVATOR APARTMENTS           : 382
09 COOPS - WALKUP APARTMENTS               : 2767
10 COOPS - ELEVATOR APARTMENTS             : 12902
11A CONDO-RENTALS                           : 43
12 CONDOS - WALKUP APARTMENTS              : 926
13 CONDOS - ELEVATOR APARTMENTS            : 12989
14 RENTALS - 4-10 UNIT                     : 671
15 CONDOS - 2-10 UNIT RESIDENTIAL          : 1281
16 CONDOS - 2-10 UNIT WITH COMMERCIAL UNIT : 96
17 CONDO COOPS                             : 1201
22 STORE BUILDINGS                         : 935
37 RELIGIOUS FACILITIES                    : 100
42 CONDO CULTURAL/MEDICAL/EDUCATIONAL/ETC  : 13
46 CONDO STORE BUILDINGS                   : 154
47 CONDO NON-BUSINESS STORAGE              : 377
01 ONE FAMILY DWELLINGS                    : 18235
02 TWO FAMILY DWELLINGS                    : 15828
03 THREE FAMILY DWELLINGS                  : 4384
04 TAX CLASS 1 CONDOS                      : 1656
21 OFFIC

In [8]:
# List of commercial property categories with their associated numbers
commercial_categories = [
    "22 STORE BUILDINGS",
    "21 OFFICE BUILDINGS",
    "29 COMMERCIAL GARAGES",
    "43 CONDO OFFICE BUILDINGS",
    "28 COMMERCIAL CONDOS",
    "31 COMMERCIAL VACANT LAND",
    "49 CONDO WAREHOUSES/FACTORY/INDUS",
    "30 WAREHOUSES",
    "45 CONDO HOTELS",
    "27 FACTORIES",
    "34 THEATRES",
    "25 LUXURY HOTELS",
    "26 OTHER HOTELS"
]

# Filter the DataFrame to include only the rows with the specified commercial categories
commercial_df = df[df['BUILDING CLASS CATEGORY'].isin(commercial_categories)]

# Display the new DataFrame
print(commercial_df)

       Unnamed: 0  BOROUGH   NEIGHBORHOOD    BUILDING CLASS CATEGORY  \
195           199        1  ALPHABET CITY         22 STORE BUILDINGS   
928           932        1        CHELSEA        21 OFFICE BUILDINGS   
929           933        1        CHELSEA        21 OFFICE BUILDINGS   
930           934        1        CHELSEA        21 OFFICE BUILDINGS   
931           935        1        CHELSEA        21 OFFICE BUILDINGS   
...           ...      ...            ...                        ...   
84177        8043        5    WESTERLEIGH         22 STORE BUILDINGS   
84178        8044        5    WESTERLEIGH         22 STORE BUILDINGS   
84179        8045        5    WESTERLEIGH  31 COMMERCIAL VACANT LAND   
84392        8258        5    WILLOWBROOK         22 STORE BUILDINGS   
84546        8412        5        WOODROW         22 STORE BUILDINGS   

      TAX CLASS AT PRESENT  BLOCK  LOT EASE-MENT BUILDING CLASS AT PRESENT  \
195                      4    390   34                   